## メモ
前処理内容
- 欠損値のあるデータを削除（データ数 : 48842 => 45222）
- カテゴリカル変数を one hot encoding（特徴量数 : 15 => 105）

作成したデータは"pre_adult.csv"として保存

c_charge_desc sex-race, age-cat 多すぎるのでなくして良い

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)

In [4]:
data = pd.read_table('original/compas.csv', sep=',')
#data = data.rename(columns={20 : 'label'})
data.head()

,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,two_year_recid,sex-race
0,Male,69,Greater than 45,Other,0,0,0,0,F,Aggravated Assault w/Firearm,0,Male-Other
1,Male,34,25 - 45,African-American,0,0,0,0,F,Felony Battery w/Prior Convict,1,Male-African-American
2,Male,24,Less than 25,African-American,0,0,1,4,F,Possession of Cocaine,1,Male-African-American
3,Male,44,25 - 45,Other,0,0,0,0,M,Battery,0,Male-Other
4,Male,41,25 - 45,Caucasian,0,0,0,14,F,Possession Burglary Tools,1,Male-Caucasian


In [7]:
print(data.isnull().sum())
print()
print("COMPAS data has no missing value")

sex                0
age                0
age_cat            0
race               0
juv_fel_count      0
juv_misd_count     0
juv_other_count    0
priors_count       0
c_charge_degree    0
c_charge_desc      0
two_year_recid     0
sex-race           0
dtype: int64

COMPAS data has no missing value


In [11]:
print(data.info())
print()
print("age-cat, race, c_charge_degree, c_dharge_desc and sex-race are categorical features")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sex              6167 non-null   object
 1   age              6167 non-null   int64 
 2   age_cat          6167 non-null   object
 3   race             6167 non-null   object
 4   juv_fel_count    6167 non-null   int64 
 5   juv_misd_count   6167 non-null   int64 
 6   juv_other_count  6167 non-null   int64 
 7   priors_count     6167 non-null   int64 
 8   c_charge_degree  6167 non-null   object
 9   c_charge_desc    6167 non-null   object
 10  two_year_recid   6167 non-null   int64 
 11  sex-race         6167 non-null   object
dtypes: int64(6), object(6)
memory usage: 578.3+ KB
None

age-cat, race, c_charge_degree, c_dharge_desc and sex-race are categorical features


In [10]:
print(data.nunique())

sex                  2
age                 65
age_cat              3
race                 6
juv_fel_count       10
juv_misd_count      10
juv_other_count      9
priors_count        36
c_charge_degree      2
c_charge_desc      389
two_year_recid       2
sex-race            12
dtype: int64


In [5]:
def binarized(df):
    #Binarization 
    df_bin = pd.DataFrame()
    df_bin['race:Caucasian']=(df['race_Caucasian'])
    df_bin['race:AfricanAmerican']=(df['race_African-American'])
    df_bin['race:Asian']=(df['race_Asian'])
    df_bin['race:Hispanic']=(df['race_Hispanic'])
    df_bin['race:Other']=(df['race_Other'])
    df_bin['race:NativeAmerican']=(df['race_Native American'])

    #sex
    df_bin['gender:Male']=(df['sex']==1).apply(int)
    df_bin['gender:Female']=(df['sex']==0).apply(int)

    #age
    df_bin['age:25-45']=(df['age_cat_25 - 45'])
    df_bin['age:>45']=(df['age_cat_Greater than 45'])
    df_bin['age:<25']=(df['age_cat_Less than 25'])


    #degree_charge
    df_bin['charge:Felony']=(df['c_charge_degree_F'])
    df_bin['charge:Misdemeanor']=(df['c_charge_degree_M'])
    
    #juv_fel_count
    df_bin['juv_fel_count:<=1.5']=(df.juv_fel_count <= 1.5).apply(int)
    df_bin['juv_fel_count:>1.5']=(df.juv_fel_count > 1.5).apply(int)

    #juv_other_count
    df_bin['juv_other_count:<=0.5']=(df.juv_other_count <= 0.5).apply(int)
    df_bin['juv_other_count:>0.5']=(df.juv_other_count > 0.5).apply(int)

    #priors_count
    df_bin['priors_count:<1']=(df.priors_count <= 0.5).apply(int)
    df_bin['priors_count:0.5-1.5']=((0.5 < df.priors_count) & (df.priors_count <= 1.5)).apply(int)
    df_bin['priors_count:1.5-2.5']=((1.5 < df.priors_count) & (df.priors_count <= 2.5)).apply(int)
    df_bin['priors_count:2.5-4.5']=((2.5 < df.priors_count) & (df.priors_count <= 4.5)).apply(int)
    df_bin['priors_count:4.5-5.5']=((4.5 < df.priors_count) & (df.priors_count <= 5.5)).apply(int)
    df_bin['priors_count:5.5-8.5']=((5.5 < df.priors_count) & (df.priors_count <= 8.5)).apply(int)
    df_bin['priors_count:8.5-14.5']=((8.5 < df.priors_count) & (df.priors_count <= 14.5)).apply(int)
    df_bin['priors_count:14.5-15.5']=((14.5 < df.priors_count) & (df.priors_count <= 15.5)).apply(int)
    df_bin['priors_count:15.5-27.5']=((15.5 < df.priors_count) & (df.priors_count <= 27.5)).apply(int)
    df_bin['priors_count:>27.5']=(df.priors_count > 27.5).apply(int)

    
    # end binarized

    scores = pd.DataFrame()
    scores['two_year_recid'] = (df['two_year_recid']==0).apply(int)
    features = df_bin

    data_bin=df_bin.values
    data_bin=data_bin.astype(int)
    df=df.reset_index(drop=True)
        
    #Getting labels
    Labels=[0,1]
    label_bin=np.zeros((2,len(data_bin)))

    label_bin[1,:]=((df['two_year_recid']==1).apply(int)).values
    label_bin[0,:]=((df['two_year_recid']==0).apply(int)).values
    
    label_bin=label_bin.T
    label_bin=label_bin.astype(int)

    return df_bin, data_bin, label_bin, Labels, features, scores